In [ ]:
# ================================
# SIMPLE: Sales per Unit by Category
# Output: SQL Table → category_sales_per_unit
# Perfect for filtering in Power BI / Excel
# ================================

import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# CONFIGURATION
# -----------------------------
CONFIG = {
    'sql_query': """
        SELECT 
            CategoryName,
            Qty AS UnitsSold,
            NetSales AS Revenue
        FROM _Gift_from_Carpenters 
        WHERE NetSales IS NOT NULL
        AND Qty > 0
        AND CategoryName IS NOT NULL
    """,
    'server': '100.99.225.51',
    'database': 'TEST1',
    'username': 'SA',
    'password': 'a31536000',
    'driver': '{ODBC Driver 17 for SQL Server}',
    'target_table': 'category_sales_per_unit'
}

# -----------------------------
# 1. LOAD DATA
# -----------------------------
def load_data():
    conn_str = (
        f"DRIVER={CONFIG['driver']};"
        f"SERVER={CONFIG['server']};"
        f"DATABASE={CONFIG['database']};"
        f"UID={CONFIG['username']};"
        f"PWD={CONFIG['password']};"
        f"Encrypt=yes;TrustServerCertificate=yes"
    )
    print("Connecting to database...")
    conn = pyodbc.connect(conn_str)
    df = pd.read_sql(CONFIG['sql_query'], conn)
    conn.close()
    print(f"Loaded {len(df):,} rows")
    return df

# -----------------------------
# 2. CALCULATE SALES PER UNIT
# -----------------------------
def calculate_sales_per_unit(df):
    result = df.groupby('CategoryName').agg(
        TotalRevenue=('Revenue', 'sum'),
        TotalUnits=('UnitsSold', 'sum')
    ).reset_index()

    result['SalesPerUnit'] = (result['TotalRevenue'] / result['TotalUnits']).round(2)
    result['RunDate'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Keep only what you need
    result = result[['CategoryName', 'TotalRevenue', 'TotalUnits', 'SalesPerUnit', 'RunDate']]
    
    print(f"Calculated for {len(result)} categories")
    return result

# -----------------------------
# 3. SAVE TO SQL
# -----------------------------
def save_to_sql(df):
    engine = create_engine(
        f"mssql+pyodbc://{CONFIG['username']}:{CONFIG['password']}@"
        f"{CONFIG['server']}/{CONFIG['database']}?"
        f"driver=ODBC+Driver+17+for+SQL+Server&Encrypt=yes&TrustServerCertificate=yes"
    )
    df.to_sql(CONFIG['target_table'], engine, if_exists='replace', index=False)
    print(f"Data saved to SQL table: {CONFIG['target_table']}")

# -----------------------------
# MAIN
# -----------------------------
def main():
    print("Starting Sales per Unit by Category...\n")
    df = load_data()
    result = calculate_sales_per_unit(df)
    save_to_sql(result)
    
    print("\n" + "="*50)
    print("DONE!")
    print("="*50)
    print("Table: category_sales_per_unit")
    print("Columns: CategoryName, TotalRevenue, TotalUnits, SalesPerUnit, RunDate")
    print("Ready for Power BI, Excel, or any filter!")
    print("="*50)

if __name__ == "__main__":
    main()

Starting Sales per Unit by Category...

Connecting to database...
Loaded 27,152 rows
Calculated for 221 categories
Data saved to SQL table: category_sales_per_unit

DONE!
Table: category_sales_per_unit
Columns: CategoryName, TotalRevenue, TotalUnits, SalesPerUnit, RunDate
Ready for Power BI, Excel, or any filter!
